<a href="https://colab.research.google.com/github/Rajeevku1/AIAgentSamples/blob/main/ChatusingGradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import os
import requests
import gradio as gr
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import userdata
import json
import gradio as gr


In [9]:
openai_api_key = userdata.get('OPENAI_API_KEY')
anthropic_api_key = userdata.get('ANTHROPIC_API_KEY')
TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')

# Initialize OpenAI clients
if openai_api_key:
    openai_client = OpenAI(api_key=openai_api_key)
else:
    openai_client = None

anthropic_url = "https://api.anthropic.com/v1/"
if anthropic_api_key:
    anthropic_client = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
else:
    anthropic_client = None

In [10]:
system_message = "You are a helpful assistant that responds in markdown without code blocks"

In [11]:
def stream_claude(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = anthropic_client.chat.completions.create(
        model="claude-sonnet-4-5-20250929",
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
    return result

In [12]:
def stream_model(prompt, model):
    if model=="GPT":
        result = stream_gpt(prompt, model)
    elif model=="Claude":
        result = stream_claude(prompt)
    elif model=="GPT Reasoning":
        result = stream_gpt(prompt, model)
    elif model=="GPT Internet":
        result = stream_gpt(prompt, model)
    else:
        raise ValueError("Unknown model")
    return result

In [46]:
def stream_gpt(prompt,model):
    if not openai_client:
        return "Error: OpenAI API key not set"

    system_message = "You are a helpful assistant can use internet to answer queries"
    messages = [{"role": "system", "content": system_message}, {"role": "user", "content": prompt}]
    if model=="GPT Reasoning":
        response = openai_client.chat.completions.create(model="gpt-5-mini", messages=messages, reasoning_effort="minimal")
    elif model=="GPT Internet":
        promptForInternet = ask_with_internet(prompt)
        messages = [{"role": "user", "content": promptForInternet}]
        #print(messages)
        response = openai_client.chat.completions.create(model="gpt-4o-mini", messages=messages)
    else:
        response = openai_client.chat.completions.create(model="gpt-4", messages=messages)
    return response.choices[0].message.content

In [50]:
def chatWithAIusingGrido():
    # Let's use Markdown
    # Are you wondering why it makes any difference to set system_message when it's not referred to in the code below it?
    # I'm taking advantage of system_message being a global variable, used back in the message_gpt function (go take a look)
    # Not a great software engineering practice, but quite common during Jupyter Lab R&D!

    system_message = "You are a helpful assistant that responds in markdown without code blocks"
    message_input = gr.Textbox(label="Your message:", info="Enter a message", lines=7)
    model_Name  = gr.Dropdown(["GPT", "GPT Reasoning","Claude", "GPT Internet"],label="Select model", value="GPT")
    message_output = gr.Textbox(label="Response:", lines=12)

    view = gr.Interface(
        fn=stream_model,
        title="AI Chat Bot",
        inputs=[message_input,model_Name],
        outputs=[message_output],
        examples=[
            ["Explain the Transformer architecture to a layperson", "GPT" ],
            ["Explain the Transformer architecture to an aspiring AI engineer", "Claude"]
            ],
        flagging_mode="never"
        )
    view.launch(inbrowser=True)

In [47]:
def tavily_search(query):
    url = "https://api.tavily.com/search"
    payload = {
        "api_key": TAVILY_API_KEY,
        "query": query,
        "max_results": 3
    }
    return requests.post(url, json=payload).json()



In [48]:
def ask_with_internet(question):
    # Step 1: Search
    results = tavily_search(question)

    # Step 2: Extract text
    context = "\n".join([r["content"] for r in results["results"]])

    # Step 3: Ask LLM with context
    prompt = f"""
    Use the following web search results to answer the question.

    Search Results:
    {context}

    Question: {question}

    Answer using the search results.
    """
    #print(prompt)
    return prompt


In [ ]:
prompt = ask_with_internet("who is president of USA")
response = stream_gpt(prompt, "GPT Internet")
print(response)

In [51]:
chatWithAIusingGrido()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b442afe307a2bdeff1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
